In [153]:
import json
import pandas as pd
import os
from os.path import basename, splitext

In [154]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

In [155]:
RM = 1.5
GM = 0.8

In [156]:
def load_workforce(workforce_file, OC, RM, GM, model, max_n_shifts=None):
    with open(workforce_file, 'r') as file:
        data = json.load(file)
        data_df = pd.DataFrame(data)
        df_workforce = pd.DataFrame(data)
    df_workforce = df_workforce[(df_workforce['outsourcing_cost_multiplier']==OC)&(df_workforce['regional_multiplier']==RM)&(df_workforce['global_multiplier']==GM)]
    #fixed or flex
    if model in ['fixed','flex']:
        df_workforce = df_workforce[df_workforce['model']==model]
    #partflex
    else:
        df_workforce = df_workforce[(df_workforce['model']==model)&(df_workforce['max_n_shifts']==max_n_shifts)]
    df_workforce.reset_index(drop = True, inplace=True)
    return df_workforce, data_df


In [159]:
for base_file in list_city_dt:
    results = pd.DataFrame()
    workforce_excel = pd.read_excel(f'../workforce_size/{base_file}_all_trials.xlsx')
    for model in ['flex',  'partflex', 'fixed']:
            if model == 'partflex':
                for max_n_shifts in [4, 3, 2]:
                    previous_value = 0
                    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:

                        # Load Json
                        workforce_json, data_df = load_workforce(f'../workforce_size/{base_file}.json', OC, RM, GM, model, max_n_shifts)

                        # Subset the excel file
                        subset_excel = workforce_excel[(workforce_excel['outsourcing_cost_multiplier']==OC)\
                                                    &(workforce_excel['region_multiplier']==RM)\
                                                        &(workforce_excel['global_multiplier']==GM)\
                                                            &(workforce_excel['model']==model)\
                                                              &(workforce_excel['max_n_shifts']==max_n_shifts)]

                        # Rounding the objective values to two decimal places 
                        subset_excel.loc[:, 'objective_value'] = subset_excel['objective_value'].round(5)

                        # Finding the minimum objective value rounded to two decimal places
                        maximum_obj_val = subset_excel['objective_value'].min()

                        # Finding the workforce size corresponding to the minimum objective value
                        workforce_size = subset_excel[subset_excel['objective_value'] == maximum_obj_val]['workforce_size_trial'].min() + 1

                        if workforce_size < previous_value:
                            workforce_size = previous_value
                        print(workforce_size)
                        
                        # Update the Json 
                        data_df.loc[
                            (data_df['outsourcing_cost_multiplier'] == OC) &
                            (data_df['regional_multiplier'] == RM) &
                            (data_df['global_multiplier'] == GM) &
                            (data_df['model'] == model) &
                            (data_df['max_n_shifts'] == max_n_shifts),
                            'workforce_size_region0_new'
                        ] = workforce_size
                    
                        dict_out = {} 
                        for col in data_df.columns: 
                             dict_out[col]=data_df[col].tolist()

                        # Save the updated Json
                        with open(f'../workforce_size/{base_file}_updated_2.json', 'w') as file:
                            json.dump(dict_out, file, indent=4)

                        # Make results table
                        results_temp = pd.DataFrame({'outsourcing_cost_multiplier': OC, 'regional_multiplier': RM, 'global_multiplier': GM, 'model': model, 'max_n_shifts':max_n_shifts, 'workforce_size': workforce_size}, index=[0])
                        results = pd.concat([results, results_temp])

                        # Check if workforce size is increasing
                        if results['workforce_size'].diff().values[-1] < 0 and OC != 1.2:
                            print(f'Error: Workforce size is decreasing for OC = {OC}, model = {model}, base_file = {base_file}, max_n_shifts = {max_n_shifts}')
                        previous_value = workforce_size
            else:
                previous_value = 0
                for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
                
                    # Load Json
                    workforce_json, data_df = load_workforce(f'../workforce_size/{base_file}.json', OC, RM, GM, model)

                    # Subset the excel file
                    subset_excel = workforce_excel[(workforce_excel['outsourcing_cost_multiplier']==OC)\
                                                &(workforce_excel['region_multiplier']==RM)\
                                                    &(workforce_excel['global_multiplier']==GM)\
                                                        &(workforce_excel['model']==model)]

                    # Rounding the objective values to two decimal places 
                    subset_excel.loc[:, 'objective_value'] = subset_excel['objective_value'].round(5)

                    # Finding the minimum objective value rounded to two decimal places
                    maximum_obj_val = subset_excel['objective_value'].min()

                    # Finding the workforce size corresponding to the minimum objective value
                    workforce_size = subset_excel[subset_excel['objective_value'] == maximum_obj_val]['workforce_size_trial'].min() + 1

                    if workforce_size < previous_value:
                            workforce_size = previous_value
                    print(workforce_size)
                    # Update the Json 
                    data_df.loc[
                        (data_df['outsourcing_cost_multiplier'] == OC) &
                        (data_df['regional_multiplier'] == RM) &
                        (data_df['global_multiplier'] == GM) &
                        (data_df['model'] == model),
                        'workforce_size_region0_new'
                    ] = workforce_size

                    dict_out = {} 
                    for col in data_df.columns: 
                            dict_out[col]=data_df[col].tolist()
                    # Save the updated Json
                    with open(f'../workforce_size/{base_file}_updated_2.json', 'w') as file:
                       json.dump(dict_out, file, indent=4)
         

                    # Make results table
                    results_temp = pd.DataFrame({'outsourcing_cost_multiplier': OC, 'regional_multiplier': RM, 'global_multiplier': GM, 'model': model, 'workforce_size': workforce_size}, index=[0])
                    results = pd.concat([results, results_temp])

                    # Check if workforce size is increasing
                    if results['workforce_size'].diff().values[-1] < 0 and OC != 1.2:
                        print(f'Error: Workforce size is decreasing for OC = {OC}, model = {model}, base_file = {base_file}')
                    previous_value = workforce_size
    results.to_excel(f'../workforce_size/{base_file}_workforce_size.xlsx', index=False)


20
21
21
21
23
20
21
21
21
23
20
21
21
21
23
20
21
21
21
25
23
25
25
25
25


28
31
31
31
31
28
30
30
30
30
28
28
28
28
28
28
31
32
33
35
28
31
32
33
35
47
49
50
52
55
47
49
50
52
55
47
49
50
52
55
54
58
60
60
62
54
58
60
60
62
44
44
44
44
44
44
44
44
44
44
44
44
44
44
44
98
102
107
109
115
98
102
107
109
115
13
15
17
17
19
13
15
17
17
19
13
15
17
17
19
13
15
17
20
21
15
17
20
20
21
19
22
23
23
24
19
22
23
23
24
19
19
23
23
24
22
22
22
22
24
22
22
22
22
24
33
33
33
36
36
33
33
33
36
36
33
33
33
36
36
37
39
41
41
43
37
39
41
41
43
52
56
56
56
56
52
56
56
56
56
52
56
56
56
56
67
70
72
74
78
67
70
72
74
78
12
14
15
15
15
12
14
15
15
15
12
14
15
15
15
15
15
15
15
16
15
15
15
15
16
22
22
24
24
26
22
22
24
24
26
22
22
24
24
26
26
26
26
26
27
26
26
26
26
27
38
42
42
42
44
38
42
42
42
44
38
42
42
42
44
45
47
48
49
52
45
47
48
49
52
73
76
77
81
84
73
76
77
81
84
73
76
77
81
84
87
92
94
94
99
87
92
94
94
99
27
29
30
30
31
27
29
30
30
31
27
29
30
30
31
32
33
34
34
36
32
33
34
34
36
52
52
54
55
55
52
52
54
55
55
52
52
54
55
55
57
60
61
63
65
57
60
61
63
65
92
95
99
99
100
9

In [160]:
data_df.sort_values(by=['model', 'max_n_shifts', 'outsourcing_cost_multiplier', 'regional_multiplier', 'global_multiplier'], inplace=True)
data_df

,instance_file_base,city,demand_baseline,model,max_n_shifts,outsourcing_cost_multiplier,regional_multiplier,global_multiplier,workforce_size_region0,objective_value_prev,objective_value,objective_value_next,workforce_size_region0_new
0,paris_db=4.00,paris,4.0,fixed,NaN,1.2,1.5,0.8,208,NaN,61728.367168,61728.367168,NaN
5,paris_db=4.00,paris,4.0,fixed,NaN,1.5,1.5,0.8,216,NaN,75876.144021,75876.144021,NaN
10,paris_db=4.00,paris,4.0,fixed,NaN,1.8,1.5,0.8,225,NaN,89986.493813,89986.493813,NaN
15,paris_db=4.00,paris,4.0,fixed,NaN,2.0,1.5,0.8,229,NaN,99373.941375,99373.941375,NaN
20,paris_db=4.00,paris,4.0,fixed,NaN,2.5,1.5,0.8,240,NaN,122796.789454,122796.789454,240.0
1,paris_db=4.00,paris,4.0,flex,NaN,1.2,1.5,0.8,178,NaN,60685.730665,60687.787905,NaN
6,paris_db=4.00,paris,4.0,flex,NaN,1.5,1.5,0.8,183,NaN,74760.069922,74760.879153,NaN
11,paris_db=4.00,paris,4.0,flex,NaN,1.8,1.5,0.8,184,NaN,88819.870140,88820.050059,NaN
16,paris_db=4.00,paris,4.0,flex,NaN,2.0,1.5,0.8,186,NaN,98180.904198,98185.507137,NaN
21,paris_db=4.00,paris,4.0,flex,NaN,2.5,1.5,0.8,191,NaN,121566.913802,121567.627695,NaN


In [ ]:
with open('../workforce_size/paris_db=0.50_updated_2.json', 'r') as file:
        data = json.load(file)
        data_df = pd.DataFrame(data)
data_df.sort_values(by=['model', 'max_n_shifts', 'outsourcing_cost_multiplier', 'regional_multiplier', 'global_multiplier'], inplace=True)
data_df

,instance_file_base,city,demand_baseline,model,max_n_shifts,outsourcing_cost_multiplier,regional_multiplier,global_multiplier,workforce_size_region0,objective_value_prev,objective_value,objective_value_next,workforce_size_region0_new
0,paris_db=0.50,paris,0.5,fixed,NaN,1.2,1.5,0.8,34,NaN,7520.429619,7520.429619,NaN
5,paris_db=0.50,paris,0.5,fixed,NaN,1.5,1.5,0.8,33,NaN,9194.949524,9194.949524,NaN
10,paris_db=0.50,paris,0.5,fixed,NaN,1.8,1.5,0.8,36,NaN,10866.019429,10866.019429,NaN
15,paris_db=0.50,paris,0.5,fixed,NaN,2.0,1.5,0.8,34,NaN,11978.661587,11978.661587,NaN
20,paris_db=0.50,paris,0.5,fixed,NaN,2.5,1.5,0.8,37,NaN,14752.936111,14752.936111,36.0
1,paris_db=0.50,paris,0.5,flex,NaN,1.2,1.5,0.8,27,NaN,7345.612381,7345.912381,NaN
6,paris_db=0.50,paris,0.5,flex,NaN,1.5,1.5,0.8,30,NaN,9003.798808,9004.257976,NaN
11,paris_db=0.50,paris,0.5,flex,NaN,1.8,1.5,0.8,30,NaN,10658.433571,10658.433571,NaN
16,paris_db=0.50,paris,0.5,flex,NaN,2.0,1.5,0.8,30,NaN,11759.121746,11759.121746,NaN
21,paris_db=0.50,paris,0.5,flex,NaN,2.5,1.5,0.8,31,NaN,14503.152183,14503.152183,NaN
